In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch

def print_gpu_utilization():
    if torch.cuda.is_available():
        used_memory = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU 메모리 사용량: {used_memory:.3f} GB")
    else:
        print("런타임 유형을 GPU로 변경하세요")

print_gpu_utilization()

GPU 메모리 사용량: 0.000 GB


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    print_gpu_utilization()
    return model, tokenizer

model_id = "EleutherAI/polyglot-ko-1.3b"
model, tokenizer = load_model_and_tokenizer(model_id)
print("모델 파라미터 데이터 타입: ", model.dtype)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.575 GB
모델 파라미터 데이터 타입:  torch.float16


In [4]:
from transformers import AdamW
from torch.utils.data import DataLoader

def estimate_memory_of_gradients(model):
    total_memory = 0
    for param in model.parameters():
        if param.grad is not None:
            total_memory += param.grad.nelement() * param.grad.element_size()
    return total_memory

def estimate_memory_of_optimizer(optimizer):
    total_memory = 0
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                total_memory += v.nelement() * v.element_size()
    return total_memory

In [5]:
def train_model(model, dataset, training_args):
    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())
    model.train()
    gpu_utilization_printed = False
    for step, batch in enumerate(train_dataloader, start=1):
        batch = {k: v.to(model.device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / training_args.gradient_accumulation_steps
        loss.backward()

        if step % training_args.gradient_accumulation_steps == 0:
            optimizer.step()
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)
            if not gpu_utilization_printed:
                print_gpu_utilization()
                gpu_utilization_printed = True
            optimizer.zero_grad()

    print(f"옵티마이저 상태의 메모리 사용량: {optimizer_memory / (1024 ** 3):.3f} GB")
    print(f"그레디언트 메모리 사용량: {gradients_memory / (1024 ** 3):.3f} GB")

In [6]:
%%capture
!pip install datasets

In [7]:
import numpy as np
from datasets import Dataset

def make_dummy_dataset():
  seq_len, dataset_size = 256, 64
  dummy_data = {
      "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
      "labels": np.random.randint(100, 30000, (dataset_size, seq_len)),
  }
  dataset = Dataset.from_dict(dummy_data)
  dataset.set_format("pt")
  return dataset

In [8]:
import gc

def cleanup():
    if 'model' in globals():
        del globals()['model']
    if 'dataset' in globals():
        del globals()['dataset']
    gc.collect()
    torch.cuda.empty_cache()

In [9]:
from transformers import TrainingArguments, Trainer

def gpu_memory_experiment(batch_size,
                          gradient_accumulation_steps=1,
                          gradient_checkpointing=False,
                          model_id="EleutherAI/polyglot-ko-1.3b",
                          peft=None):

    print(f"배치 사이즈: {batch_size}")
    model, tokenizer = load_model_and_tokenizer(model_id, peft=peft)
    if gradient_checkpointing == True or peft == 'qlora':
        model.config.use_cache = False

    dataset = make_dummy_dataset()

    training_args = TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        gradient_checkpointing=gradient_checkpointing,
        output_dir="./result",
        num_train_epochs=1
      )

    try:
        train_model(model, dataset, training_args)
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(e)
        else:
            raise e
    finally:
        del model, dataset
        gc.collect()
        torch.cuda.empty_cache()
        print_gpu_utilization()

In [10]:
cleanup()
print_gpu_utilization()

for batch_size in [4, 8, 16]:
    gpu_memory_experiment(batch_size)

    torch.cuda.empty_cache()

GPU 메모리 사용량: 2.460 GB
배치 사이즈: 4


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.575 GB
GPU 메모리 사용량: 10.469 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB
배치 사이즈: 8


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.591 GB
GPU 메모리 사용량: 10.902 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.591 GB
GPU 메모리 사용량: 11.766 GB
CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 31.06 MiB is free. Process 56487 has 14.71 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 57.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
GPU 메모리 사용량: 0.016 GB


In [11]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=4, gradient_accumulation_steps=4)

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 4


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.591 GB
GPU 메모리 사용량: 10.469 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB


In [12]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16)

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.591 GB
GPU 메모리 사용량: 11.766 GB
CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 31.06 MiB is free. Process 56487 has 14.71 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 57.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
GPU 메모리 사용량: 0.016 GB


In [13]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, gradient_checkpointing=True)

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.591 GB
GPU 메모리 사용량: 10.266 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    elif peft == 'lora':
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )

        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    print_gpu_utilization()
    return model, tokenizer

In [15]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, peft='lora')

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.1180
GPU 메모리 사용량: 2.597 GB
GPU 메모리 사용량: 4.345 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레디언트 메모리 사용량: 0.006 GB
GPU 메모리 사용량: 0.016 GB


In [16]:
from scipy.stats import norm

def quantile_normal(p):
  return norm.ppf(p)

print(quantile_normal(0.5))
print(quantile_normal(0.6))

0.0
0.2533471031357997


In [17]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-dav058yg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-dav058yg
  Resolved https://github.com/huggingface/transformers.git to commit e5fd865ebae062b7cf03a81b8c6affeb39f30bec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [18]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    elif peft == 'lora':
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )

        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
    elif peft == 'qlora':
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )
        bnb_config = BitsAndBytesConfig(
                  load_in_4bit=True,
                  bnb_4bit_use_double_quant=True,
                  bnb_4bit_quant_type="nf4",
                  bnb_4bit_compute_dtype=torch.float16
              )
        model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model)
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    print_gpu_utilization()
    return model, tokenizer

In [20]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, peft='qlora')

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.922 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.1180
GPU 메모리 사용량: 2.065 GB
GPU 메모리 사용량: 2.604 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레디언트 메모리 사용량: 0.006 GB
GPU 메모리 사용량: 0.922 GB
